Movie recommendation system

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
#User ratings dataframe
ratings=pd.read_csv("./ml-100k/u.data",sep="\t",names=["user_id","movie_id","rating","timestamp"])
ratings.head()

In [ ]:
#Movies dataframe
columns=["movie_id","title","release_date","video_release_date","imdb_url","unknown",
         "Action","Adventure","Animation","Children\'s","Comedy","Crime","Documentary",
         "Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi",
         "Thriller","War","Western"]
movies=pd.read_csv("./ml-100k/u.item",sep="|",names=columns,encoding='latin1')
print(movies.head())

In [ ]:
#Users dataframe
users=pd.read_csv("./ml-100k/u.user",sep="|",names=["user_id","age","gender","occupation","zip_code"])
print(users.head())

In [ ]:
print(ratings.info())

In [ ]:
print(movies.info())

In [ ]:
print(users.info())

In [ ]:
ratings.isnull().sum()

In [ ]:
users.isnull().sum()

In [ ]:
movies.isnull().sum()

In [ ]:
print(movies.loc[movies["release_date"].isnull()])

In [ ]:
print(movies[movies["title"]=="unknown"])
movies=movies[movies["title"]!="unknown"]
print("Removed movie with unknown title, genre and release date")

In [ ]:
movies.drop(columns=["video_release_date"], inplace=True)   #remove video release date as all are null

In [ ]:
print(movies.isnull().sum())

In [ ]:
movies["imdb_url"]=movies["imdb_url"].fillna(value="none")   #Fill the missing url with a value none
movies.isnull().sum()

In [ ]:
movies.info()

In [ ]:
movies["release_date"]=pd.to_datetime(movies["release_date"])
print(movies["release_date"].head())

In [ ]:
movies.info()

In [ ]:
ratings[~ratings["movie_id"].isin(movies["movie_id"])]  #ratings for the movie that was removed

In [ ]:
ratings=ratings[ratings["movie_id"].isin(movies["movie_id"])]
ratings.loc[ratings["movie_id"]==267]   #Empty row means successful deletion of ratings for the deleted movie

Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

In [ ]:
# Ratings distribution
ratings['rating'].value_counts().sort_index().plot(kind='bar')
plt.title('Rating Distribution')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.show()


In [ ]:
#Ratings per user
ratings_per_user = ratings.groupby('user_id').size().sort_index()
ratings_per_user.plot(kind='hist', bins=50)
plt.title('Ratings per User')
plt.xlabel('Number of Ratings')
plt.show()

In [ ]:
# Ratings per movie
ratings_per_movie = ratings.groupby('movie_id').size()
ratings_per_movie.plot(kind='hist', bins=50)
plt.title('Ratings per Movie')
plt.xlabel('Number of Ratings')
plt.show()

In [ ]:
# Basic statistics of ratings dataframe
print(ratings.isnull().sum())
print("*"*50)
print(ratings.info())
print("*"*50)
print(ratings.describe())

In [ ]:
print("Exploring the movies dataframe")
genre_columns=movies.columns[4:]
print("Genre columns => ",genre_columns)

In [ ]:
# Sum of movies in each genre (A movie can belong to several genre)
genre_counts = movies[genre_columns].sum().sort_values().plot(kind="bar")

In [ ]:
ratings_per_movie = ratings.groupby('movie_id').size()

plt.figure(figsize=(10,6))
ratings_per_movie.plot(kind='hist', bins=50, color='skyblue', edgecolor='black')
plt.title('Ratings per Movie')
plt.xlabel('Number of Ratings')
plt.ylabel('Number of Movies')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Join with ratings count
ratings_count = ratings.groupby('movie_id').size().reset_index(name='count')
top_movies = ratings_count.sort_values(by='count', ascending=False).head(10)

# Merge to get titles
top_movies = pd.merge(top_movies, movies, on='movie_id')
print(top_movies[['title', 'count']])


In [ ]:
# Merge ratings with movie titles
merged_df = pd.merge(ratings, movies, on='movie_id')

# Show first few rows
merged_df.head()

In [ ]:
top_rated = merged_df['title'].value_counts().head(10)
print(top_rated)

In [ ]:
top_10 = merged_df['title'].value_counts().head(10)

In [ ]:
plt.figure(figsize=(10, 6))
top_10.plot(kind='barh', color='skyblue', edgecolor='black')
plt.xlabel('Number of Ratings')
plt.title('Top 10 Most Rated Movies')
plt.gca().invert_yaxis()  # Most rated at top
plt.tight_layout()
plt.show()

In [ ]:
#Feature Engineering – Ratings per Month

# 2. Convert timestamp to datetime
ratings['datetime'] = pd.to_datetime(ratings['timestamp'], unit='s')

# 3. Extract year-month (e.g., '2020-05')
ratings['year_month'] = ratings['datetime'].dt.to_period('M')

# 4. Count ratings per movie each month
monthly_counts = ratings.groupby(['movie_id', 'year_month']).size().reset_index(name='ratings_count')

# 5. Pivot to wide format (each movie, month as column)
monthly_pivot = monthly_counts.pivot(
    index='movie_id',
    columns='year_month',
    values='ratings_count'
).fillna(0)

# 6. (Optional) Add a feature: total ratings across all months
monthly_pivot['total_ratings'] = monthly_pivot.sum(axis=1)

feat = pd.merge(monthly_pivot.reset_index(), movies, on='movie_id')
print(feat.head())


In [ ]:
#Line Plot of Movie Ratings Over Time
# (Assuming 'monthly_counts' DataFrame from earlier)
# Example: plot ratings count over time for a single movie_id
movie_id = 50  # change as needed
ts = monthly_counts[monthly_counts.movie_id == movie_id]
ts.set_index('year_month', inplace=True)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(ts.index.to_timestamp(), ts['ratings_count'], marker='o')

ax.set_title(f'Monthly Ratings Count for Movie {movie_id}')
ax.set_xlabel('Month')
ax.set_ylabel('Number of Ratings')

# Format x-axis ticks every 3 months
locator = mdates.MonthLocator(interval=3)
formatter = mdates.DateFormatter('%Y-%m')
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)
fig.autofmt_xdate()

ax.grid(True)
plt.show()


In [ ]:
#Heatmap of Ratings by Movie and Time
# Pivot into matrix: movies x months
pivot = monthly_counts.pivot(
    index='movie_id',
    columns='year_month',
    values='ratings_count'
).fillna(0)

# (Optionally select top-10 most active movies for clarity)
top_movies = pivot.sum(axis=1).sort_values(ascending=False).head(10).index
pivot_top = pivot.loc[top_movies]

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_top, cmap='YlGnBu', cbar_kws={'label': 'Ratings Count'})
plt.title('Ratings Count per Month (Top 10 Movies)')
plt.xlabel('Year-Month')
plt.ylabel('Movie ID')
plt.show()


In [ ]:
#Detecting Abnormal Movie Rating Patterns Using Isolation Forest.
from sklearn.ensemble import IsolationForest

# Build a feature df
features = merged_df.groupby('movie_id').agg({
    'rating': ['count', 'mean']
}).reset_index()
features.columns = ['movie_id', 'rating_count', 'avg_rating']

model = IsolationForest(contamination=0.02, random_state=42)
model.fit(features[['rating_count', 'avg_rating']])

features['anomaly'] = model.predict(features[['rating_count', 'avg_rating']])
outliers_iso = features[features['anomaly'] == -1]
print("IsolationForest outliers:", outliers_iso)

# Plot outliers
plt.figure(figsize=(8,6))
plt.scatter(features['rating_count'], features['avg_rating'], c=(features['anomaly']==-1), cmap='coolwarm', alpha=0.7)
plt.xlabel('Rating Count')
plt.ylabel('Average Rating')
plt.title('Outliers via Isolation Forest')
plt.show()


In [ ]:
### Simple Item-Based Collaborative Filtering (Slope One)
from collections import defaultdict

# Build user→item ratings dict
user_ratings = defaultdict(dict)
for row in ratings.itertuples():
    user_ratings[row.user_id][row.movie_id] = row.rating

# Compute deviations and counts
dev, count = defaultdict(lambda: defaultdict(int)), defaultdict(lambda: defaultdict(int))
for u, items in user_ratings.items():
    for i, ri in items.items():
        for j, rj in items.items():
            if i != j:
                dev[i][j] += ri - rj
                count[i][j] += 1

# Average deviations
for i in dev:
    for j in dev[i]:
        dev[i][j] /= count[i][j]

# Predict function
def slope_one_predict(user_id, item):
    num, den = 0.0, 0
    for j, ruj in user_ratings[user_id].items():
        if item in dev[j]:
            num += (dev[j][item] + ruj) * count[j][item]
            den += count[j][item]
    return num / den if den else None

# Example: Predict ratings for a sample user
test_user = ratings.user_id.unique()[0]
preds = {item: slope_one_predict(test_user, item)
         for item in ratings.movie_id.unique()
         if item not in user_ratings[test_user]}
top_preds = sorted(preds.items(), key=lambda x: x[1] or 0, reverse=True)[:10]

print("Top 10 recommendations (item: predicted rating):", top_preds)



In [ ]:
### Matrix Factorization with Truncated SVD (NumPy & Scikit‑learn)
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Split train/test
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

# Pivot to matrix
train_mat = train.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
test_mat = test.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
train_mat.head()


In [ ]:
# Factorize with truncated SVD
U, Sigma, VT = randomized_svd(train_mat.values, n_components=20, random_state=42)
Sigma_mat = np.diag(Sigma)

# Reconstruct train prediction matrix
pred_train = np.dot(np.dot(U, Sigma_mat), VT)
train_mat.head()
test_mat.head()
train_mat.shape
test_mat.shape

In [ ]:
# Perform truncated SVD to reconstruct user–item ratings matrix (low-rank approximation)

# Split
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

# Collect all users/items
users = sorted(ratings['user_id'].unique())
items = sorted(ratings['movie_id'].unique())

# Pivot training matrix
train_mat = train.pivot_table(index='user_id', columns='movie_id',
                              values='rating', fill_value=0).reindex(
    index=users, columns=items, fill_value=0
)

# Apply TruncatedSVD
tsvd = TruncatedSVD(n_components=20, random_state=42)
train_reduced = tsvd.fit_transform(train_mat)

# Reconstruct full matrix
pred_matrix = np.dot(train_reduced, tsvd.components_)

# Wrap into DataFrame
pred_ratings = pd.DataFrame(pred_matrix, index=users, columns=items)

# Evaluate on test
preds, truths = [], []
for u, i, true_val in test[['user_id', 'movie_id', 'rating']].itertuples(index=False):
    preds.append(pred_ratings.at[u, i])
    truths.append(true_val)

rmse = np.sqrt(mean_squared_error(truths, preds))
mae = mean_absolute_error(truths, preds)
print(f"SVD RMSE = {rmse:.3f}, MAE = {mae:.3f}")


Content Based Filtering

In [ ]:
movies.columns

In [ ]:
genre_columns = movies.columns[4:]
movies['genre'] = movies[genre_columns].apply(
    lambda row: '|'.join([genre for genre in genre_columns if row[genre] == 1]),
    axis=1
)
movies=movies[['movie_id', 'title', 'imdb_url','genre']]


In [ ]:
movies.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer(token_pattern=r"(?u)\b[\w-]+\b")  
tfidf_matrix = tfidf.fit_transform(movies['genre'])

In [ ]:
def recommend_content_based(movie_title, movies_df, tfidf_matrix, top_n=5):
    
    # Find the input movie
    input_movie = movies_df[movies_df['title'].str.contains(movie_title, case=False)]
    if input_movie.empty:
        return pd.DataFrame(columns=['movie_id', 'title', 'similarity'])
    
    # Calculate similarities
    input_idx = input_movie.index[0]
    cosine_sim = cosine_similarity(
        tfidf_matrix[input_idx:input_idx+1], 
        tfidf_matrix
    ).flatten()
    
    # Get top-N most similar (excluding input movie)
    movies_df['similarity'] = cosine_sim
    recommendations = movies_df[movies_df['movie_id'] != input_movie.iloc[0]['movie_id']]
    
    return recommendations[['movie_id', 'title', 'similarity']].sort_values('similarity', ascending=False).head(top_n)


In [ ]:
test_movie = "toy story"  
content_recs = recommend_content_based(
    movie_title=test_movie,
    movies_df=movies,          
    tfidf_matrix=tfidf_matrix  
)
print(content_recs)

NMF Model

In [ ]:
user_item_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating')

In [ ]:
from sklearn.decomposition import NMF
user_item_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating')
# Fill NaNs with 0s (important for NMF)
user_item_matrix_filled = user_item_matrix.fillna(0)

# Train the NMF model
nmf_model = NMF(n_components=20, init='random', random_state=42, max_iter=1000)
W = nmf_model.fit_transform(user_item_matrix_filled)  # User-feature matrix
H = nmf_model.components_                             # Feature-item matrix


In [ ]:
from sklearn.preprocessing import MinMaxScaler
def recommend_nmf(user_id, user_item_matrix, nmf_model, movies_df, top_n=5):
    if user_id not in user_item_matrix.index:
        return f"User ID {user_id} not found."

    # Fill missing with 0s just like during training
    user_vector = user_item_matrix.loc[user_id].fillna(0).values.reshape(1, -1)

    # Project this user into latent space
    W_user = nmf_model.transform(user_vector)
    pred_ratings = np.dot(W_user, nmf_model.components_).flatten()

    # Create DataFrame of predictions
    recommendations = pd.DataFrame({
        'movie_id': user_item_matrix.columns,
        'nmf_score': pred_ratings
    })

    # Exclude movies already rated
    rated_movies = user_item_matrix.loc[user_id].dropna().index
    recommendations = recommendations[~recommendations['movie_id'].isin(rated_movies)]

    # Normalize scores
    recommendations['nmf_score'] = MinMaxScaler().fit_transform(recommendations[['nmf_score']])

    # Join with movie titles
    final_recommendations = recommendations.merge(
        movies_df[['movie_id', 'title']], on='movie_id'
    ).sort_values(by='nmf_score', ascending=False)

    return final_recommendations[['movie_id', 'title', 'nmf_score']].head(top_n)


In [ ]:
recommend_nmf(
    user_id=5,
    user_item_matrix=user_item_matrix,
    nmf_model=nmf_model,
    movies_df=movies,
    top_n=5
)


In [ ]:
def hybrid_recommender(user_id=None,
                       movie_title=None,
                       ratings_df=None,
                       user_item_matrix=None,
                       nmf_model=None,
                       movies_df=None,
                       tfidf_matrix=None,
                       alpha=0.5,
                       top_n=5):
    cb_recs = pd.DataFrame()
    cf_recs = pd.DataFrame()

    # Content-Based Recommendations
    if movie_title:
        cb_recs = recommend_content_based(
            movie_title=movie_title,
            movies_df=movies_df,
            tfidf_matrix=tfidf_matrix,
            top_n=top_n
        )
        cb_recs = cb_recs.rename(columns={'similarity': 'cb_score'})

    # Collaborative Recommendations
    if user_id is not None:
        cf_recs = recommend_nmf(
            user_id=user_id,
            user_item_matrix=user_item_matrix,
            nmf_model=nmf_model,
            movies_df=movies_df,
            top_n=top_n * 2  # Use more for blending
        )
        cf_recs = cf_recs.rename(columns={'nmf_score': 'cf_score'})

    # Combine
    if not cb_recs.empty and not cf_recs.empty:
        combined = pd.merge(cb_recs, cf_recs, on=['movie_id', 'title'], how='outer').fillna(0)
        combined['score'] = alpha * combined['cf_score'] + (1 - alpha) * combined['cb_score']
        return combined.sort_values('score', ascending=False).head(top_n)[['movie_id', 'title', 'score']]

    elif not cb_recs.empty:
        cb_recs['score'] = cb_recs['cb_score']
        return cb_recs[['movie_id', 'title', 'score']]

    elif not cf_recs.empty:
        cf_recs['score'] = cf_recs['cf_score']
        return cf_recs[['movie_id', 'title', 'score']]

    else:
        return pd.DataFrame(columns=['movie_id', 'title', 'score'])


In [ ]:
hybrid_recommender(
    user_id=12,
    ratings_df=ratings,
    user_item_matrix=user_item_matrix,
    nmf_model=nmf_model,
    movies_df=movies,
    tfidf_matrix=tfidf_matrix
)


In [ ]:
hybrid_recommender(
    movie_title="star wars",
    movies_df=movies,
    tfidf_matrix=tfidf_matrix
)


In [ ]:
hybrid_recommender(
    user_id=14,
    movie_title="star wars",
    ratings_df=ratings,
    user_item_matrix=user_item_matrix,
    nmf_model=nmf_model,
    movies_df=movies,
    tfidf_matrix=tfidf_matrix,
    alpha=0.5
)


In [ ]:
import pickle

# Save DataFrames
movies.to_pickle("models/movies.pkl")
ratings.to_pickle("models/ratings.pkl")
user_item_matrix.to_pickle("models/user_item_matrix.pkl")

# Save TF-IDF vectorizer and matrix
with open("models/tfidf.pkl", "wb") as f:
    pickle.dump(tfidf, f)

with open("models/tfidf_matrix.pkl", "wb") as f:
    pickle.dump(tfidf_matrix, f)

# Save NMF model
with open("models/nmf_model.pkl", "wb") as f:
    pickle.dump(nmf_model, f)
